# Battery Lifetime Calculator

## How to use this document

If you are viewing this on github.com, this document is read only.  Download it and open it
using Jupyter Notebook if you want to edit it.

When viewing in Jupyter, click the run button on each block to step through the calculations.   

Some of the blocks have values that you can change before you click run.

## What does this document do

You have a battery, and you want to draw some power from it.   How long will it last?

Battery capacity is measured in Ampere-Hours, that is Amps multiplied Hours.   Its a literal trade-off---draw a lot of amps, you only get a few hours; draw a fraction of an amp, and you get lots of hours.  A 10Ah battery can theoretically deliver 10 Amps for 1 hour, one Amp for 10 hours, or 500mA for 20 hours.   In reality, the relationship might not be so *linear*.

The rated capacity from the manufacturer is usually for a 20 hour discharge, so if you are going to run the battery for a shorter or longer time, you may experience a different effective capacity.    Also, resellers lie outrageously about capacity.

If your load is constant (eg you draw 200mA all the time, then a theoretical lifetime is simply `capacity / current`, eg `10Ah / 200mA => 50 hours`.

But what if you need more life?  Then we need to go to low-power (sleep mode) when possible and only consume our **peak draw** for brief intervals.  The rest of the time we consume a far smaller **idle draw**

We'll use Python and the *Pint* unit library to help us model this scenario.

In [71]:
# Bring in the unit conversion library
import pint
units = pint.UnitRegistry()

## Enter your battery capacity

The rated capacity presumes you use all the power in the battery. Running many kinds of battery dead flat can damage them, so lets choose a safety factor. We'll say we only run the battery, say, 80% flat.

In [72]:
rated_capacity = 120 * units.Ah

depth_of_discharge = 0.8

# Don't change anything below here in this block
capacity = rated_capacity * depth_of_discharge
print("If we are nice to our battery we can use up to {} of its rated {} capacity".format(capacity,rated_capacity))

If we are nice to our battery we can use up to 96.0 ampere_hour of its rated 120 ampere_hour capacity


## Enter your power consumption

We're going to presume that you use full power at peak times, but in between we can go to a lower power idle mode.

The worst case is you draw this all the time, hopefully we will do better.

In [73]:
peak_draw = 500 * units.mA

idle_draw = 80 * units.uA

# Don't change anything below here in this block
print("We'll use {0} when idle but at peak times we use up to {1}".format(idle_draw,peak_draw))

We'll use 80 microampere when idle but at peak times we use up to 500 milliampere


### Enter your peak use characteristics

In [85]:
peak_duration = 10 * units.second
peaks_per_day = 64

# Don't change anything below here in this block
peak_time_per_day = (peak_duration * peaks_per_day).to(units.minute)
print("With {}x {} peaks each day, we spend {:.3} each day using our peak power of {}".format(
    peaks_per_day, peak_duration, peak_time_per_day, peak_draw))

With 64x 10 second peaks each day, we spend 10.7 minute each day using our peak power of 500 milliampere


## Calculate the worst case

Let's presume that you draw your `peak_draw` all the time, then the theoretical runtime is simply capacity divided by peak:

In [75]:
# You don't need to edit anything in this block
worst_case_lifetime = (capacity / peak_draw).to(units.hour)
print("If we use our peak power of {} all the time, we would only last for {}".format(
    peak_draw, worst_case_lifetime))

If we use our peak power of 500 milliampere all the time, we would only last for 192.0 hour


## Calculate the average daily energy use

Our average daily consumption will be the peak consumption multipled by the fraction of time we are in peak, plus the idle consumption multiplied by the remaining time

In [76]:
# You don't need to edit anything in this block
idle_time_per_day = ((24 * units.hour) - peak_time_per_day).to(units.hour)
peak_consumption_per_day = peak_draw * peak_time_per_day
idle_consumption_per_day = idle_draw * idle_time_per_day
average_daily_energy = (peak_consumption_per_day + idle_consumption_per_day).to(units.Ah) / units.day
print("With {} peaks per day, our average daily energy use is {:.3}".format(peaks_per_day,average_daily_energy))

With 30 peaks per day, our average daily energy use is 0.0436 ampere_hour / day


## Caculate the theoretical best case

Now we know how much energy we use per day, we can use the (not really) linear relationship of amps * hours to work out how long we would run at that average draw.

In reality, there is efficiency, non-linearity and self-discharge to consider, so this figure may not be precise.



In [84]:
# You don't need to edit anything in this block
best_case_lifetime = (capacity / average_daily_energy) # .to(units.day)
print("Best case lifetime is {:.5} between charges (that's about {:.2})".format(best_case_lifetime,best_case_lifetime.to(units.year)))

Best case lifetime is 2202.8 day between charges (that's about 6.0 year)
